In [ ]:
import sys
sys.version

In [ ]:

!nvidia-smi

Sun Apr 28 09:45:45 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   66C    P8              15W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install -q pypdf python-dotenv

In [ ]:
!pip -q install git+https://github.com/huggingface/transformers

In [ ]:
!pip install -q datasets loralib sentencepiece

In [ ]:
!pip install -q einops accelerate langchain bitsandbytes

In [ ]:
!pip install sentence_transformers

In [ ]:
!pip install llama-index

In [ ]:
!pip install -q chromadb

In [ ]:
!pip install llama-index-llms-huggingface

In [ ]:
!pip install llama-index-embeddings-huggingface


In [ ]:
!pip install llama-index-vector-stores-chroma

In [ ]:
!pip install llama-index-llms-ollama

In [ ]:
!pip install llama-index-embeddings-ollama

In [ ]:
from llama_index.core import SimpleDirectoryReader, ServiceContext, StorageContext, VectorStoreIndex

from llama_index.llms.huggingface import HuggingFaceLLM

from llama_index.embeddings.huggingface import HuggingFaceEmbedding

from llama_index.vector_stores.chroma import ChromaVectorStore

from llama_index.core.prompts.prompts import SimpleInputPrompt

from llama_index.core.prompts.prompts import SimpleInputPrompt

import chromadb

from IPython.display import Markdown, display

In [ ]:
chroma_client = chromadb.PersistentClient()
chroma_collection = chroma_client.get_or_create_collection('OncoAIFlow')
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
documents = SimpleDirectoryReader("/content/drive/MyDrive/OncoAIFlow").load_data()

In [ ]:
from llama_index.core.prompts.prompts import SimpleInputPrompt
system_prompt = "Tu es un assistant medical. En te basant sur le chapitre 7 qui concerne les traitements, tu dois repondre uniquement par oui ou par non. si tu n'est pas sur de toi, alors reponds par oui."
# This will wrap the default prompts that are internal to llama-index
query_wrapper_prompt = "<|USER|>{query_str}<|ASSISTANT|>"

In [ ]:
from huggingface_hub import login
login()

In [ ]:
# !huggingface-cli login

In [ ]:
!huggingface-cli whoami

In [ ]:
import torch
torch.set_default_device('cuda')

In [ ]:
llm = HuggingFaceLLM(
    context_window=8000,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.01, "do_sample": True},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="mistralai/Mistral-7B-Instruct-v0.1",
    model_name="mistralai/Mistral-7B-Instruct-v0.1",
    device_map="auto",
    tokenizer_kwargs={"max_length": 8000},
    model_kwargs={"torch_dtype": torch.float16}

)

In [ ]:
embed_model = HuggingFaceEmbedding(model_name="mixedbread-ai/mxbai-embed-large-v1")

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/171 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/113k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/677 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [ ]:
service_context = ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model
)

In [ ]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context, storage_context=storage_context)

In [ ]:
query_engine = index.as_query_engine()
response = query_engine.query("Est-ce que le traitement systemique fait partie des traitements possibles pour un patient un performance status a 4, porteur d'un CHC unifocal, avec une maladie hepatique CHILD A")
display(Markdown(f"<b>{response}</b>"))

In [ ]:
import random

# Liste de valeurs pour les balises
performance = ['0', '1', '2', '3', '4']
traitement = ['la transplantation', 'la resection', 'la destruction percutanee', 'la radiotherapie', 'le traitement systemique', 'la chimio-embolisation', 'la radiotherapie interne selective']
nodule = ['avec un nodule', 'avec deux nodules', 'avec trois nodules', 'avec de multiples nodules', 'metastatique']
maladie_hepatique = ['sur foie non cirrhotique','avec cirrhose CHILD A', 'avec cirrhose CHILD B', 'avec cirrhose CHILD C']



# Fonction pour générer un exemple aléatoire
def generer_prompt():
    performance_aleatoire = random.choice(performance)
    traitement_aleatoire = random.choice(traitement)
    nodule_aleatoire = random.choice(nodule)
    maladie_hepatique_aleatoire = random.choice(maladie_hepatique)
    return f"Est-ce que {traitement_aleatoire} est un traitement envisageable pour un patient un performance status a {performance_aleatoire}, porteur d'un CHC {nodule_aleatoire}, {maladie_hepatique_aleatoire}."

# Exemple d'utilisation
for _ in range(5):
    print(generer_prompt())


Est-ce que la resection est un traitement envisageable pour un patient un performance status a 3, porteur d'un CHC avec de multiples nodules, sur foie non cirrhotique.
Est-ce que la destruction percutanee est un traitement envisageable pour un patient un performance status a 2, porteur d'un CHC avec de multiples nodules, sur foie non cirrhotique.
Est-ce que la destruction percutanee est un traitement envisageable pour un patient un performance status a 2, porteur d'un CHC avec deux nodules, avec cirrhose CHILD C.
Est-ce que la destruction percutanee est un traitement envisageable pour un patient un performance status a 0, porteur d'un CHC avec deux nodules, avec cirrhose CHILD C.
Est-ce que la resection est un traitement envisageable pour un patient un performance status a 4, porteur d'un CHC avec deux nodules, avec cirrhose CHILD A.


In [ ]:
# Liste de valeurs pour les balises
performance = ['0', '1', '2', '3', '4']
#traitements = ['la transplantation', 'la resection', 'la destruction percutanee', 'la radiotherapie', 'le traitement systemique', 'la chimio-embolisation', 'la radiotherapie interne selective']
stade = ['a un stade localise avec un nodule', 'a un stade localise avec deux nodules', 'a un stade localise avec trois nodules', 'a un stade localise avec de multiples nodules', 'de stade metastatique']
traitement_anterieur = ['sans traitement prealable', 'avec antecedant de resection', 'avec antecedant de chimio embolisation', 'avec antecedant de destruction percutanee', 'avec antecedant de radiotherapie externe']
maladie_hepatique = ['avec un foie non cirrhotique','avec une cirrhose CHILD A', 'avec une cirrhose CHILD B', 'avec une cirrhose CHILD C']
traitements = [
    'la transplantation',
    'la resection',
    'la destruction percutanee',
    'la radiotherapie',
    'le traitement systemique',
    'la chimio-embolisation',
    'la radiotherapie interne selective'
]
# Dictionnaire associant chaque traitement à une liste de spécialistes
specialistes_par_traitement = {
    'la transplantation': ['hepatologue', 'chirurgien'],
    'la resection': ['chirurgien'],
    'la destruction percutanee': ['radiologue interventionnel'],
    'la radiotherapie': ['oncologue', 'radiologue', 'radiotherapeute'],
    'le traitement systemique': ['oncologue'],
    'la chimio-embolisation': ['radiologue interventionnel'],
    'la radiotherapie interne selective': ['radiologue', 'oncologue', 'radiotherapeute']
}

In [ ]:
import numpy as np
def flatten_extend(matrix):
  flat_list = []
  for row in matrix:
    flat_list.extend(row)
  return flat_list
specialists = np.unique(flatten_extend([specialist for specialist in specialistes_par_traitement.values()])).tolist()

In [ ]:
response = query_engine.query("Est-ce que la resection chirurgicale fait partie des traitements possibles pour un patient un performance status a 1, porteur d'un CHC unifocal, avec une maladie hepatique CHILD A")
display(Markdown(f"<b>{response}</b>"))

In [ ]:
import pandas as pd
pd.__version__

In [ ]:
# Fonction pour générer un exemple aléatoire
def generer_prompt_patient():
    performance_aleatoire = random.choice(performance)
    traitement_anterieur_aleatoire = random.choice(traitement_anterieur)
    nodule_aleatoire = random.choice(stade)
    maladie_hepatique_aleatoire = random.choice(maladie_hepatique)
    return f"patient avec un performance status a {performance_aleatoire}, porteur d'un CHC {nodule_aleatoire}, {traitement_anterieur_aleatoire} {maladie_hepatique_aleatoire}."
def generer_prompt_traitement(type_traitement,prompt_patient):
    return f"Est-ce que {type_traitement} est un traitement envisageable pour un {prompt_patient}"
def verified_response(prompt):
    while True:
      response = query_engine.query(prompt)
      # print(response)
      if str(response).lower().replace(".","") in ["oui", "non"]:
        print(str(prompt)+" "+str(response))
        return str(response).lower().replace(".","")
def lister_specialistes(prompt_patient, id_patient):
    df = pd.DataFrame(
        np.array([[prompt_patient] + ["Non" for specialist in specialists]]),
        index = [id_patient],
        columns= ['prompt'] + [specialist for specialist in specialists])
    # display(df)
    for type_traitement in traitements:
      prompt = generer_prompt_traitement(type_traitement,prompt_patient)
      # print(prompt)
      if verified_response(prompt) == "oui":
        # print(specialistes_par_traitement.get(type_traitement,[]))
        for specialist in specialistes_par_traitement.get(type_traitement,[]):
          # print(df.loc[id_patient, str(specialist)])
          df.loc[id_patient, specialist] = "Oui"
    return df

In [ ]:
lister_specialistes("Test", 1)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


,prompt,chirurgien,hepatologue,oncologue,radiologue,radiologue interventionnel,radiotherapeute
1,Test,Oui,Oui,Oui,Oui,Oui,Oui


In [ ]:
df = pd.DataFrame()
for patient_id in range(10):

  prompt_patient = generer_prompt_patient()
  new_row = lister_specialistes(prompt_patient, patient_id)
  df = pd.concat([df, new_row], axis=0)
  display(df)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Est-ce que la transplantation est un traitement envisageable pour un patient avec un performance status a 3, porteur d'un CHC a un stade localise avec un nodule, sans traitement prealable avec une cirrhose CHILD B. Non.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Est-ce que la resection est un traitement envisageable pour un patient avec un performance status a 3, porteur d'un CHC a un stade localise avec un nodule, sans traitement prealable avec une cirrhose CHILD B. Non.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Est-ce que la radiotherapie est un traitement envisageable pour un patient avec un performance status a 3, porteur d'un CHC a un stade localise avec trois nodules, avec antecedant de destruction percutanee avec une cirrhose CHILD A. Oui.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Est-ce que le traitement systemique est un traitement envisageable pour un patient avec un performance status a 3, porteur d'un CHC a un stade localise avec trois nodules, avec antecedant de destruction percutanee avec une cirrhose CHILD A. Non.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
df.to_csv("results.csv")

In [ ]:
# #results = []
# import csv
# with open('/content/drive/MyDrive/specialistes.csv', 'w', newline='') as csvfile:
#     fieldnames = ['prompt'] + [specialist for specialist in specialists]
#     print(fieldnames)
#     writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
#     writer.writeheader()
#     for patient_id in range(10):
#         prompt_patient = generer_prompt_patient()
#         QA = lister_specialistes(prompt_patient)
#         print(QA)
#         writer.writerow(QA)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['prompt', 'chirurgien', 'hepatologue', 'oncologue', 'radiologue', 'radiologue interventionnel', 'radiotherapeute']
Est-ce que la transplantation est un traitement envisageable pour un patient avec un performance status a 2, porteur d'un CHC a un stade localise avec de multiples nodules, avec antecedant de destruction percutanee avec une cirrhose CHILD B.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Non.
Est-ce que la resection est un traitement envisageable pour un patient avec un performance status a 2, porteur d'un CHC a un stade localise avec de multiples nodules, avec antecedant de destruction percutanee avec une cirrhose CHILD B.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Non.
Est-ce que la chimio-embolisation est un traitement envisageable pour un patient avec un performance status a 2, porteur d'un CHC a un stade localise avec de multiples nodules, avec antecedant de destruction percutanee avec une cirrhose CHILD B.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Non.
Est-ce que la radiotherapie interne selective est un traitement envisageable pour un patient avec un performance status a 2, porteur d'un CHC a un stade localise avec de multiples nodules, avec antecedant de destruction percutanee avec une cirrhose CHILD B.
Oui.
["patient avec un performance status a 2, porteur d'un CHC a un stade localise avec de multiples nodules, avec antecedant de destruction percutanee avec une cirrhose CHILD B."]


AttributeError: 'list' object has no attribute 'keys'

In [ ]:
pd.read_csv("result2.csv")

,prompt,answer
